In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import librosa
import numpy as np
from sklearn.mixture import GaussianMixture
import scipy.stats
import pandas as pd


In [3]:
import os
import glob
import librosa
import numpy as np
from sklearn.mixture import GaussianMixture
import pandas as pd

data_folder = "/content/drive/MyDrive/KAGGLE-2/SpeechCommand"
subfolders = [f.path for f in os.scandir(data_folder) if f.is_dir()]

Class_mapping = {'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'bed': 4, 'happy': 5, 'go': 6, 'dog': 7, 'no': 8,
                 'wow': 9, 'nine': 10, 'left': 11, 'stop': 12, 'three': 13, 'sheila': 14, 'one': 15, 'bird': 16,
                 'zero': 17, 'seven': 18, 'up': 19, 'marvin': 20, 'two': 21, 'house': 22, 'down': 23, 'six': 24,
                 'yes': 25, 'on': 26, 'five': 27, 'off': 28, 'four': 29}

order = 37
fs = 16000
frame_period = 5



Streaming output truncated to the last 5000 lines.
<ipython-input-3-be5dfd9d0b8e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append({'ID': i, 'TARGET': predicted_word}, ignore_index=True)
<ipython-input-3-be5dfd9d0b8e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append({'ID': i, 'TARGET': predicted_word}, ignore_index=True)
<ipython-input-3-be5dfd9d0b8e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append({'ID': i, 'TARGET': predicted_word}, ignore_index=True)
<ipython-input-3-be5dfd9d0b8e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.

Number of features for training: 67628


<ipython-input-3-be5dfd9d0b8e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append({'ID': i, 'TARGET': predicted_word}, ignore_index=True)


In [ ]:
# Function to extract features from a single audio file
def extract_features(file_path):
    x, fs = librosa.load(file_path)
    x = librosa.effects.preemphasis(x)

    # Extract chroma and spectral contrast features
    chroma = librosa.feature.chroma_stft(y=x, sr=fs)
    spectral_contrast = librosa.feature.spectral_contrast(y=x, sr=fs)

    # MFCC features
    mfcc = librosa.feature.mfcc(y=x, sr=fs, n_mfcc=17)
    delta = librosa.feature.delta(mfcc)
    delta_delta = librosa.feature.delta(mfcc, order=2)

    # Combine all features
    combined_features = np.vstack([mfcc, delta, delta_delta, chroma, spectral_contrast])

    return combined_features.T

features = []
labels = []
dict1 = {}
dict2 = {}



In [ ]:
# Function to load data and labels
def load_data_labels(path):
    i = 0
    for word in os.listdir(path):
        if word != "_background_noise_":
            dict1[word] = i
            dict2[i] = word
            i += 1
            feat = []
            files_path = os.path.join(path, word)

            for file in os.listdir(files_path):
                file_path = os.path.join(files_path, file)

                if file.endswith(".wav"):
                    feat.extend(extract_features(file_path))
                    labels.append(Class_mapping[word])

            features.append(feat)

    return features, labels



In [ ]:
# Load training data
features, labels = load_data_labels("/content/drive/MyDrive/KAGGLE-2/SpeechCommand")

# Train Gaussian Mixture Models
models = []
for i in range(30):
    gm = GaussianMixture(n_components=1, random_state=0).fit(features[i])
    models.append(gm)

# Load and predict on the test set
test_files = []
df = pd.read_csv("test.csv")
test_files = list(df['AUDIO_FILE'])
outputs = {'ID': [], 'TARGET': []}
df_final = pd.DataFrame(outputs)
test_path = "/content/drive/MyDrive/KAGGLE-2/SpeechCommandTest/"
for i, test_file in enumerate(test_files):
    t_path = os.path.join(test_path, test_file)
    test_features = extract_features(t_path)
    likelihoods = [model.score(test_features) for model in models]
    best_model_index = np.argmax(likelihoods)
    predicted_word = Class_mapping[dict2[best_model_index]]
    df_final = df_final.append({'ID': i, 'TARGET': predicted_word}, ignore_index=True)

df_final.to_csv("predicted.csv", index=False)

# Assuming your features list is a list of flattened combined features
num_features_for_training = len(features[0])
print("Number of features for training:", num_features_for_training)
